#### Reinforcement Learning &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Homework #2
# &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;The $\lambda$ Return

## Description

Given an MDP and a particular time step $t$ of a task (continuing or
episodic), the $\lambda$-return, $G_t^\lambda$, $0\leq\lambda\leq 1$, is
a weighted combination of the $n$-step returns $G_{t:t+n}$, $n \geq 1$:

$${G_t^\lambda = \sum\limits_{n=1}^\infty(1-\lambda)\lambda^{n-1}G_{t:t+n}.}$$

While the $n$-step return $G_{t:t+n}$ can be viewed as the target of
an $n$-step TD update rule, the $\lambda$-return can be viewed as the
target of the update rule for the TD$(\lambda)$ prediction algorithm,
which you will become familiar with in project 1.

Consider the Markov reward process described by the following state
diagram and assume the agent is in state $0$ at time $t$ (also assume
the discount rate is $\gamma=1$). A Markov reward process can be thought of as
an MDP with only one action possible from each state (denoted as action $0$ in
the figure below).

![image](https://d1b10bmlvqabco.cloudfront.net/paste/jqmfo7d3watba/9e6fd83672f880704b8418728297fc077786c8907d87fec631601da9ff4c85ef/hw2.png)

## Procedure

-   You will implement your solution using the `solve()` method
    in the code below.
    
-   You will be given `p`, the probability of transitioning from state
    $0$ to state $1$, `V`, the estimate of the value function at time
    $t$, represented as a vector
    $[V(0), V(1), V(2), V(3), V(4), V(5), V(6)]$, and `rewards`, a
    vector of the rewards $[r_0, r_1, r_2, r_3, r_4, r_5, r_6]$
    corresponding to the MDP.

-   Your return value should be a value of $\lambda$,
    strictly less than 1, such that the expected value of the
    $\lambda$-return equals the expected Monte-Carlo return at time $t$.

-   Your answer must be correct to $3$ decimal places, truncated (e.g.
    3.14159265 becomes 3.141).

## Resources

The concepts explored in this homework are covered by:

-   Lecture Lesson 3: Temporal Difference Learning

-   Chapter 7 (7.1 $n$-step TD Prediction) and Chapter 12 (12.1 The
    $\lambda$-return) of
    http://incompleteideas.net/book/the-book-2nd.html

-   'Learning to Predict by the Method of Temporal Differences', R.
    Sutton, 1988

## Submission

-   The due date is indicated on the Syllabus page for this assignment.

-   Use the template code below to implement your work.

-   Please use *python 3.6.x* or more recent version and *numpy==1.18.0* or more recent version, and you can
    use any core library (i.e., anything in the Python standard library).
    No other library can be used.

In [48]:
################
# DO NOT REMOVE
# Versions
# numpy==1.18.0
# 刘兴琰
# 202264690069
################
import numpy as np

class TDAgent(object):
    def __init__(self):
        pass

    def calculate_mc_return(self, p, rewards):
        """
        Calculate the Monte Carlo return by weighting sequences of rewards based on the probability `p`.
        """
        return p * (rewards[0] + rewards[2] + rewards[4] + rewards[5] + rewards[6]) + (1 - p) * (
            rewards[1] + rewards[3] + rewards[4] + rewards[5] + rewards[6]
        )

    def calculate_lambda_return(self, p, V, rewards, lambda_):
        """
        Calculate the lambda-return by taking the weighted sum of n-step returns.
        """
        G_lambda = 0
        for n in range(1, 6):
            if n == 1:
                G_n = p * (rewards[0] + V[1]) + (1 - p) * (rewards[1] + V[2])
            elif n == 2:
                G_n = p * (rewards[0] + rewards[2] + V[3]) + (1 - p) * (rewards[1] + rewards[3] + V[3])
            elif n == 3:
                G_n = p * (rewards[0] + rewards[2] + rewards[4] + V[4]) + (1 - p) * (rewards[1] + rewards[3] + rewards[4] + V[4])
            elif n == 4:
                G_n = p * (rewards[0] + rewards[2] + rewards[4] + rewards[5] + V[5]) + (1 - p) * (rewards[1] + rewards[3] + rewards[4] + rewards[5] + V[5])
            elif n == 5:
                G_n = p * (rewards[0] + rewards[2] + rewards[4] + rewards[5] + rewards[6]) + (1 - p) * (rewards[1] + rewards[3] + rewards[4] + rewards[5] + rewards[6])

            
            G_lambda += (1 - lambda_) * (lambda_ ** (n - 1)) * G_n

        G_lambda += (lambda_ ** 5) * (p * (rewards[0] + rewards[2] + rewards[4] + rewards[5] + rewards[6]) + (1 - p) * (rewards[1] + rewards[3] + rewards[4] + rewards[5] + rewards[6]))

        return G_lambda

    def solve(self, p, V, rewards):
        """
        Find the value of lambda that minimizes the difference between the lambda-return and Monte Carlo return.
        """
        mc_return = self.calculate_mc_return(p, rewards)

        best_lambda = 0
        min_difference = float('inf')

        for lambda_ in np.arange(0, 1, 0.0001):
            lambda_return = self.calculate_lambda_return(p, V, rewards, lambda_)
            difference = abs(mc_return - lambda_return)

            if difference < min_difference:
                min_difference = difference
                best_lambda = lambda_

        return round(best_lambda, 3)

## Test cases

We have provided some test cases for you to help verify your implementation.

In [49]:
## DO NOT MODIFY THIS CODE.  This code will ensure that your submission
## will work proberly

import unittest

class TestTDNotebook(unittest.TestCase):
    def test_case_1(self):
        agent = TDAgent()
        np.testing.assert_almost_equal(
            agent.solve(
                p=0.81,
                V=[0.0, 4.0, 25.7, 0.0, 20.1, 12.2, 0.0],
                rewards=[7.9, -5.1, 2.5, -7.2, 9.0, 0.0, 1.6]
            ),
            0.622,
            decimal=3
        )
        
    def test_case_2(self):
        agent = TDAgent()
        np.testing.assert_almost_equal(
            agent.solve(
                p=0.22,
                V=[12.3, -5.2, 0.0, 25.4, 10.6, 9.2, 0.0],
                rewards=[-2.4, 0.8, 4.0, 2.5, 8.6, -6.4, 6.1]
            ),
            0.519,
            decimal=3
        )
        
    def test_case_3(self):
        agent = TDAgent()

        np.testing.assert_almost_equal(
            agent.solve(
                p=0.64,
                V=[-6.5, 4.9, 7.8, -2.3, 25.5, -10.2, 0.0],
                rewards=[-2.4, 9.6, -7.8, 0.1, 3.4, -2.1, 7.9]
            ),
            0.207,
            decimal=3
        )

unittest.main(argv=[''], verbosity=2, exit=False)

test_case_1 (__main__.TestTDNotebook.test_case_1) ... ok
test_case_2 (__main__.TestTDNotebook.test_case_2) ... ok
test_case_3 (__main__.TestTDNotebook.test_case_3) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.080s

OK
